### Problem 4

#### Import libraries  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Reshape, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets import mnist

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
np.random.seed(11)
tf.random.set_seed(13)

2022-01-20 23:35:33.331717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-20 23:35:33.331776: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Parameters  

In [2]:
used_digits = [5,7]
n_data = 1500
n_epoch = 50
n_noise = 100
batch_size = 32

img_dir = 'images'
model_g = 'model_dcgan-b{}_g.h5'.format(batch_size)
model_d = 'model_dcgan-b{}_d.h5'.format(batch_size)

#### (A) Generator  

In [3]:
def generator_model():
    model = Sequential()
    model.add(Dense(256, input_shape=(n_noise, ), activation="tanh"))
    model.add(Dense(32 * 7 * 7))
    model.add(BatchNormalization())
    model.add(Activation("tanh"))
    model.add(Reshape((7, 7, 32), input_shape=(7 * 7 * 32,)))
    model.add(UpSampling2D(size=(2, 2))) # (A1)
    model.add(Conv2D(16, (5, 5),
                     padding="same",
                     activation="tanh",
                     data_format="channels_last"))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5),
                     padding="same",
                     activation="tanh",
                     data_format="channels_last"))
    return model

#### Discriminator  

In [4]:
def discriminator_model():
    model = Sequential()
    model.add(Conv2D(16, (5, 5),
                     padding="same",
                     input_shape=(28, 28, 1),
                     activation="tanh",
                     data_format="channels_last"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (5, 5),
                     activation="tanh",
                     data_format="channels_last"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation="tanh"))
    model.add(Dense(1, activation="sigmoid"))
    return model

#### D(G(z))  

In [5]:
def generator_containing_discriminator(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

#### For output image samples  

In [6]:
def combine_images(generated_images):
    generated_images = generated_images.reshape(generated_images.shape[0],
                                                generated_images.shape[3],
                                                generated_images.shape[1],
                                                generated_images.shape[2])
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num) / width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[0, :, :]
    return image

#### (B) Training (learning)  

In [7]:
def train(BATCH_SIZE):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train[ np.isin(y_train, used_digits) ]
    y_train = y_train[ np.isin(y_train, used_digits) ]
    X_train = X_train[:n_data]
    y_train = y_train[:n_data]
    X_train = (X_train.astype(np.float32) - 127.5)/127.5 # (B1)
    X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])
    discriminator = discriminator_model()
    generator = generator_model()
    discriminator_on_generator = \
        generator_containing_discriminator(generator, discriminator)
    #d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    #g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    #generator.compile(loss="binary_crossentropy", optimizer="SGD")
    d_optim = Adam()
    g_optim = Adam()
    generator.compile(loss="binary_crossentropy", optimizer="Adam")
    discriminator_on_generator.compile(
        loss="binary_crossentropy", optimizer=g_optim)
    discriminator.trainable = True
    discriminator.compile(loss="binary_crossentropy", optimizer=d_optim)
    noise = np.zeros((BATCH_SIZE, n_noise))
    ret = []
    for epoch in range(n_epoch):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        n_index = int(X_train.shape[0]/BATCH_SIZE)
        for index in range(n_index):
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, n_noise)
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            image_batch = image_batch.reshape(image_batch.shape[0],
                                              image_batch.shape[2],
                                              image_batch.shape[3],
                                              image_batch.shape[1])
            generated_images = generator.predict(noise, verbose=0)
            if (epoch == 0 and index == 0) or index == (n_index-1):
                image = combine_images(generated_images)
                image = image*127.5+127.5
                img_file = str(epoch)+"_"+str(index)+".png"
                img_file = os.path.join(img_dir, img_file)
                Image.fromarray(image.astype(np.uint8)).save(img_file)

            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, n_noise)
            discriminator.trainable = False
            g_loss = discriminator_on_generator.train_on_batch(
                noise, [1] * BATCH_SIZE)
            discriminator.trainable = True
            loss_msg = "batch {:d}  d_loss: {:f}".format(index, d_loss)
            loss_msg += "  g_loss: {:f}".format(g_loss)
            print(loss_msg)
            ret.append((epoch, index, d_loss, g_loss))
            if index % 10 == 9:
                generator.save_weights("generator", True)
                discriminator.save_weights("discriminator", True)
                
    generator.save(model_g)
    discriminator.save(model_d)
    return ret

#### Generate images using generator  

In [8]:
def generate(BATCH_SIZE):
    generator = generator_model()
    #generator.compile(loss="binary_crossentropy", optimizer="SGD")
    generator.compile(loss="binary_crossentropy", optimizer="Adam")
    generator.load_weights("generator")
    noise = np.zeros((BATCH_SIZE, n_noise))
    for i in range(BATCH_SIZE):
        noise[i, :] = np.random.uniform(-1, 1, n_noise)
    generated_images = generator.predict(noise, verbose=1)
    image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save("generated_image.png")

#### Do training (learning)  

In [9]:
%%time

train_log = train(BATCH_SIZE=batch_size)

Instructions for updating:
Colocations handled automatically by placer.


2022-01-20 23:35:41.987544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-20 23:35:41.987603: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-20 23:35:41.987644: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anhvu): /proc/driver/nvidia/version does not exist


Epoch is 0
Number of batches 46


2022-01-20 23:35:42.741747: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


FileNotFoundError: [Errno 2] No such file or directory: 'images/0_0.png'

#### Check train_log  

In [ ]:
df_fit = pd.DataFrame(train_log)
df_fit.columns = ['epoch', 'batch', 'd_loss', 'g_loss']
display(df_fit.head())
display(df_fit.tail())

In [ ]:
plt.plot(df_fit.index, df_fit['d_loss'], label='d_loss')
plt.plot(df_fit.index, df_fit['g_loss'], label='g_loss')
plt.xlabel('Batches')
plt.ylabel('Losses')
plt.legend()
plt.show()

#### Generate image by using Generator  

In [ ]:
generate(batch_size)